# Python and C++ extension

## Importing library

In [ ]:
import sys
sys.path.append('/content/CppToPython')

In [ ]:
import numpy as np
import GeDiM4Py as gedim

### Initialize

In [ ]:
lib = gedim.ImportLibrary("/content/CppToPython/release/GeDiM4Py.so")

config = { 'GeometricTolerance': 1.0e-8 }
gedim.Initialize(config, lib)

## Elliptic Equation

Solving the following equation on square $\bar{\Omega} = [0, 1] \times [0, 1]$

$$
\begin{cases}
\nabla \cdot (a \nabla u) + b \cdot \nabla u + c u = f & \text{in } \Omega\\
a \nabla u \cdot n_1 = g_1 & \text{in } \Gamma_{left}\\
a \nabla u \cdot n_2 = g_2 & \text{in } \Gamma_{right}\\
u = 1.1 & \text{in } ∂Ω∖ (\Gamma_{left} ∪ \Gamma_{right})
\end{cases}
$$

where $u = 16 xy(1-x)(1-y) + 1.1$.

In [ ]:
def Poisson_A():
	return 10.0
def Poisson_B():
	return 0.1
def Poisson_C():
	return 2.0

def Poisson_a(numPoints, points):
	values = np.ones(numPoints) * Poisson_A()
	return values.ctypes.data

def Poisson_b(numPoints, points):
	values = np.ones((2, numPoints)) * Poisson_B()
	return values.ctypes.data

def Poisson_c(numPoints, points):
	values = np.ones(numPoints) * Poisson_C()
	return values.ctypes.data

def Poisson_f(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = Poisson_A() * 32.0 * (matPoints[1,:] * (1.0 - matPoints[1,:]) + matPoints[0,:] * (1.0 - matPoints[0,:])) + \
	Poisson_B() * 16.0 * (1.0 - 2.0 * matPoints[0,:]) * matPoints[1,:] * (1.0 - matPoints[1,:]) + \
	Poisson_B() * 16.0 * (1.0 - 2.0 * matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:]) + \
	Poisson_C() * 16.0 * (matPoints[1,:] * (1.0 - matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:])) + Poisson_C() * 1.1
	return values.ctypes.data

def Poisson_exactSolution(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = 16.0 * (matPoints[1,:] * (1.0 - matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:])) + 1.1
	return values.ctypes.data

def Poisson_exactDerivativeSolution(direction, numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)

	if direction == 0:
		values = 16.0 * (1.0 - 2.0 * matPoints[0,:]) * matPoints[1,:] * (1.0 - matPoints[1,:])
	elif direction == 1:
		values = 16.0 * (1.0 - 2.0 * matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:])
	else:
		values = np.zeros(numPoints)

	return values.ctypes.data

def Poisson_strongTerm(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = 16.0 * (matPoints[1,:] * (1.0 - matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:])) + 1.1
	return values.ctypes.data

def Poisson_weakTerm_right(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = Poisson_A() * 16.0 * (1.0 - 2.0 * matPoints[0,:]) * matPoints[1,:] * (1.0 - matPoints[1,:])
	return values.ctypes.data
	
def Poisson_weakTerm_left(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = - Poisson_A() * 16.0 * (1.0 - 2.0 * matPoints[0,:]) * matPoints[1,:] * (1.0 - matPoints[1,:])
	return values.ctypes.data

### Define Simulation Parameters

In [ ]:
meshSize = 0.1
order = 1

### Create Mesh

The library accepts the possibility to create square domains and rectangle domains.

To description of the domain borders are passed to the library for the vertices and the edges of the domain as integer values called `markers`.
Each `marker` identifies a different boundary condition.

__In this example__:

 `marker=2` identifies $Γ_{right}$, `marker=3` identifies $Γ_{left}$ and `marker=1` identifies the Dirichlet boundary.

In [ ]:
domain = { 'SquareEdge': 1.0, 'VerticesBoundaryCondition': [1,1,1,1], 'EdgesBoundaryCondition': [1,2,1,3], 'DiscretizationType': 1, 'MeshCellsMaximumArea': meshSize }
[meshInfo, mesh] = gedim.CreateDomainSquare(domain, lib)

In [ ]:
gedim.PlotMesh(mesh)

### Create Discrete Space FEM

The boundary condition types are passed to the library during the creation of the discrete space.
The types are the following:
* `BoundaryConditionType=1`: internal mesh point;
* `BoundaryConditionType=2`: strong boundary mesh point (Dirichlet in this example);
* `BoundaryConditionType=3`: weak boundary mesh point (Neumann in this example).
The array `BoundaryConditionType` describes for each `marker` the type of boundary condition associated.

__NB__: the array has dimension `num_markers+1`, as the first element is associated to non-usable `marker=0`.

__In this example__:

we have $3$ different markers, thus `BoundaryConditionsType` has size $3+1=4$. In particular `marker=1` has type `BoundaryConditionsType[1]=2` (Dirichlet), `marker=2` has type `BoundaryConditionsType[2]=3` and `marker=3` (Neumann) has type `BoundaryConditionsType[3]=3` (Neumann).

In [ ]:
discreteSpace = { 'Order': order, 'Type': 1, 'BoundaryConditionsType': [1, 2, 3, 3] }
[problemData, dofs, strongs] = gedim.Discretize(discreteSpace, lib)

In [ ]:
gedim.PlotDofs(mesh, dofs, strongs)

### Assemble linear system

In [ ]:
[stiffness, stiffnessStrong] = gedim.AssembleStiffnessMatrix(Poisson_a, problemData, lib)

[advection, advectionStrong] = gedim.AssembleAdvectionMatrix(Poisson_b, problemData, lib)

[reaction, reactionStrong] = gedim.AssembleReactionMatrix(Poisson_c, problemData, lib)

forcingTerm = gedim.AssembleForcingTerm(Poisson_f, problemData, lib)

solutionStrong = gedim.AssembleStrongSolution(Poisson_strongTerm, 1, problemData, lib)

weakTerm_right = gedim.AssembleWeakTerm(Poisson_weakTerm_right, 2, problemData, lib)
weakTerm_left = gedim.AssembleWeakTerm(Poisson_weakTerm_left, 3, problemData, lib)

### Solve linear system

In [ ]:
solution = gedim.LUSolver(stiffness + advection + reaction, \
    forcingTerm - \
    (stiffnessStrong + advectionStrong + reactionStrong) @ solutionStrong + \
    weakTerm_right + \
    weakTerm_left, lib)

In [ ]:
gedim.PlotSolution(mesh, dofs, strongs, solution, solutionStrong)

In [ ]:
gedim.ExportSolution(Poisson_exactSolution, solution, solutionStrong, lib)

### Compute errors

In [ ]:
errorL2 = gedim.ComputeErrorL2(Poisson_exactSolution, solution, solutionStrong, lib)

errorH1 = gedim.ComputeErrorH1(Poisson_exactDerivativeSolution, solution, solutionStrong, lib)

print("dofs", "h", "errorL2", "errorH1")
print(problemData['NumberDOFs'], '{:.16e}'.format(problemData['H']), '{:.16e}'.format(errorL2), '{:.16e}'.format(errorH1))

## Heat Conductivity Equation

Solving the following equation on square $\bar{\Omega} = [-1, +1] \times [-1, +1]$

$$
\begin{cases}
\nabla \cdot (k \nabla u) = 0 & \text{in } \Omega\\
k \nabla u \cdot n_1 = g & \text{in } \Gamma_{down}\\
u = 0 & \text{in } \Gamma_{up}\\
k \nabla u \cdot n_2 = 0 & \text{otherwise} 
\end{cases}
$$

where $k = k_1$ if $x^2 + y^2 \leq R^2$ and $k = 1$ otherwise. 

In [ ]:
def Heat_R():
	return 0.5
def Heat_K():
	return 6.68
def Heat_G():
	return 0.94

def Heat_k(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values = np.ones(numPoints)
	for p in range(0, numPoints):
		if (matPoints[0,p] * matPoints[0,p] + matPoints[1,p] * matPoints[1,p]) <= (Heat_R() * Heat_R() + 1.0e-16):
			values[p] = Heat_K()
	return values.ctypes.data

def Heat_weakTerm_down(numPoints, points):
	values = np.ones(numPoints) * Heat_G()
	return values.ctypes.data

def Empty_exactSolution(numPoints, points):
	values = np.zeros(numPoints)
	return values.ctypes.data

### Define Simulation Parameters

In [ ]:
order = 2

### Import Mesh

In [ ]:
%%writefile ImportMesh.csv
InputFolderPath
/content/CppToPython/Meshes/Mesh2

In [ ]:
[meshInfo, mesh] = gedim.ImportDomainMesh2D(lib)

In [ ]:
gedim.PlotMesh(mesh)

### Create Discrete Space FEM

In [ ]:
discreteSpace = { 'Order': order, 'Type': 1, 'BoundaryConditionsType': [1, 3, 3, 2] }
[problemData, dofs, strongs] = gedim.Discretize(discreteSpace, lib)

In [ ]:
gedim.PlotDofs(mesh, dofs, strongs)

### Assemble linear system

In [ ]:
[stiffness, stiffnessStrong] = gedim.AssembleStiffnessMatrix(Heat_k, problemData, lib)
	
weakTerm_down = gedim.AssembleWeakTerm(Heat_weakTerm_down, 1, problemData, lib)

### Solve linear system

In [ ]:
solution = gedim.LUSolver(stiffness, weakTerm_down, lib)

In [ ]:
gedim.PlotSolution(mesh, dofs, strongs, solution, np.zeros(problemData['NumberStrongs']))

In [ ]:
gedim.ExportSolution(Empty_exactSolution, solution, np.zeros(problemData['NumberStrongs']), lib)